<a href="https://colab.research.google.com/github/bsenst/LoopOfHenle/blob/main/DataExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://colab.research.google.com/drive/1UxZ1M0LNUxjiiImYyzObPmgc7_5DE7OH?usp=sharing

In [50]:
import pandas as pd

In [49]:
lab = pd.read_csv("LabsALL 2015-2022.csv", sep=";")
lab.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679298 entries, 0 to 8679297
Data columns (total 13 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Patient      int64  
 1   Report       int64  
 2   ID           int64  
 3   EntryDate    object 
 4   EntryTime    object 
 5   Code         object 
 6   NCLP         float64
 7   Analyte      object 
 8   ValueNumber  object 
 9   ValueText    object 
 10  RefHigh      object 
 11  RefLow       object 
 12  Unit         object 
dtypes: float64(1), int64(3), object(9)
memory usage: 860.8+ MB


In [51]:
dgs = pd.read_excel("Dg from report.xlsx")
dgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138718 entries, 0 to 138717
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Patient      138718 non-null  int64         
 1   Date         138718 non-null  datetime64[ns]
 2   mainDgCode   137984 non-null  object        
 3   OtherDgCode  127406 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 4.2+ MB


In [59]:
print(dgs.Patient.nunique(), lab.Patient.nunique())

15663 14158


In [60]:
print(dgs.Patient.unique()[:10])
print(lab.Patient.unique()[:10])

[ 209536    8451  107968  179366  189733 1178721  598938    3024 1202159
  110877]
[ 324729 1188371 1186803  235302  592170  196613  195042  223112  103484
   40544]


In [64]:
lab.isna().sum()

Patient              0
Report               0
ID                   0
EntryDate            0
EntryTime            0
Code           8106112
NCLP            573524
Analyte           1923
ValueNumber    1338881
ValueText      7340434
RefHigh        1747017
RefLow         1747253
Unit           1383102
dtype: int64

In [65]:
lab.shape

(8679298, 13)

In [66]:
lab.columns

Index(['Patient', 'Report', 'ID', 'EntryDate', 'EntryTime', 'Code', 'NCLP',
       'Analyte', 'ValueNumber', 'ValueText', 'RefHigh', 'RefLow', 'Unit'],
      dtype='object')

In [69]:
lab.head()

,Patient,Report,ID,EntryDate,EntryTime,Code,NCLP,Analyte,ValueNumber,ValueText,RefHigh,RefLow,Unit
0,324729,27200171,154214712,30.09.2022,30.12.1899 7:42:00,SZU-HEP_0004__R,NaN,HBsAg konfirmace,NaN,externě,NaN,NaN,NaN
1,324729,27200171,154214720,30.09.2022,30.12.1899 7:42:00,ZU-OKM_0024__R,NaN,Rotaviry + Noroviry stolice,NaN,odběr,NaN,NaN,NaN
2,324729,1453155,18244355,16.04.2015,30.12.1899 7:31:00,NaN,3482.0,s_vápník celk.,2.37,NaN,2.55,2.15,mmol/l
3,324729,1453155,18244356,16.04.2015,30.12.1899 7:31:00,NaN,2618.0,s_fosfor,1.32,NaN,1.23,0.71,mmol/l
4,324729,1453155,18244357,16.04.2015,30.12.1899 7:31:00,NaN,3940.0,s_hořčík,0.75,NaN,0.94,0.71,mmol/l


In [70]:
lab[lab.Analyte=="s_fosfor"]

,Patient,Report,ID,EntryDate,EntryTime,Code,NCLP,Analyte,ValueNumber,ValueText,RefHigh,RefLow,Unit
3,324729,1453155,18244356,16.04.2015,30.12.1899 7:31:00,NaN,2618.0,s_fosfor,1.32,NaN,1.23,0.71,mmol/l
62,324729,9644427,106786641,24.03.2016,30.12.1899 7:42:00,NaN,2618.0,s_fosfor,1.11,NaN,1.23,0.71,mmol/l
2544,235302,1253711,15585285,09.01.2015,30.12.1899 6:30:00,NaN,2618.0,s_fosfor,1.14,NaN,1.41,0.76,mmol/l
5047,5440,1577843,19777681,20.05.2015,30.12.1899 6:38:00,NaN,2618.0,s_fosfor,1.02,NaN,1.41,0.76,mmol/l
5066,5440,2133493,26550291,03.11.2015,30.12.1899 6:34:00,NaN,2618.0,s_fosfor,1.13,NaN,1.41,0.76,mmol/l
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8678823,335472,20271526,36331324,31.10.2018,30.12.1899 7:58:00,NaN,2618.0,s_fosfor,0.85,NaN,1.53,0.71,mmol/l
8678988,335472,18126720,11831050,21.04.2017,30.12.1899 7:17:00,NaN,2618.0,s_fosfor,1.07,NaN,1.53,0.71,mmol/l
8679070,335472,21822233,55133092,06.11.2019,30.12.1899 7:59:00,NaN,2618.0,s_fosfor,1.13,NaN,1.53,0.71,mmol/l
8679152,335472,23506430,86633984,20.11.2020,30.12.1899 7:18:00,NaN,2618.0,s_fosfor,0.8,NaN,1.53,0.71,mmol/l


In [71]:
dgs.mainDgCode.value_counts()

I10: Esenciální (primární) hypertenze                                                      12697
E11.9: Diabetes mellitus 2. typu bez komplikací                                             2232
E78.2: Smíšená hyperlipidemie                                                               2011
I25.9: Chronická ischemická choroba srdeční NS                                              1955
E11.7: Diabetes mellitus 2. typu s mnohočetnými komplikacemi                                1804
                                                                                           ...  
H46: Zánět zrakového nervu - neuritis nervi optici                                             1
B00.2: Herpetická gingivostomatitida a faryngotonsilitida původce: virus Herpes simplex        1
I25.5: Ischemická kardiomyopatie, st.p. impl. 2D ICD ze sek. prevence                          1
Z85.0: St.p. Ca rekta.                                                                         1
K76.9: Přechodná elevace jater

In [85]:
dgs.Date[dgs.Patient==8451].value_counts()

2022-01-27    23
2015-01-08    21
2022-08-08    13
2022-03-21     1
Name: Date, dtype: int64